In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_contratos = pd.read_csv('../data/terceirizados/arquivos_tratados_v2/contratos_validados_completo_v2.csv', low_memory=False, dtype='str')

In [3]:
df_contratos.head(15)

,nr_contrato,cd_orgao_siafi,nm_razao_social,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_cnpj,nm_unidade_prestacao,sg_orgao,...,id_contrato,objeto_contrato,objeto_compra,dataAssinatura,dataPublicacaoDOU,dataInicioVigencia,dataFimVigencia,tipo_fornecedor,valorInicialCompra,valorFinalCompra
0,000132016,30108,"CONSERMA - SERVICOS, MANUTENCAO E TRANSPORTES ...",MJSP,200352,SUPERINTENTENCIA REGIONAL DE POLICIA FEDERAL N...,SR/DPF/ES,00394494002503,"DRCOR/SR/PF/ES, SELOG/SR/PF/ES, GAB/SR/PF/ES, ...",DPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000082023,30802,CENTURION SEGURANCA E VIGILANCIA LTDA,MJSP,200117,SUPERINTENDENCIA DA POLICIA RODOVIARIA FEDERAL...,SRPRF-SP,67668194000179,"DEL08, DEL07, SEDE, DEL06, DEL04, DEL09, DEL05...",DPRF/MJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000082023,30802,REDENTOR SEGURANCA E VIGILANCIA EIRELI,MJSP,200129,SUPERINTENDENCIA REG. POL. RODV. FEDERAL-AL,SRPRF-AL,1696924000137,SEDE,DPRF/MJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000082023,36201,ALFOSERVICE PRESTADORA DE SERVICO LTA,MINISTER,254420,FIOCRUZ,FIOCRUZ/PRESIDENCIA,17259378000107,INSTITUTO AGGEU MAGALHAES,FIOCRUZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000082023,37202,MULTILIMP SERVICOS TERCEIRIZADOS LTDA.,MPS,510678,SUPERINTENDENCIA REGIONAL NORTE/CENTRO-OESTE,SUPER NORTE/CENTRO,16934245000126,"GERENCIA EXECUTIVA CUIABA, GERENCIA EXECUTIVA ...",INSS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,000082023,42207,"ADSERVICON – ADMINISTRACAO, SERVICOS & CONTABI...",MINC,423034,MUSEU DO OURO,REPRES.IBRAM/MG-ES,9489558000157,"MUSEU DO OURO, MUSEU REGIONAL DE SAO JOAO DEL-REI",IBRAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,000082024,25205,CNS NACIONAL DE SERVICOS LTDA,MPO,114601,CDDI,IBGE/ADM.CENTRAL/RJ,33285255000105,ENCE/GEAPO,IBGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,000082024,26280,INTERLIMP GESTAO DE SERVICOS EIRELI,MIN.EDUC,154049,FUNDACAO UNIVERSIDADE FEDERAL DE SAO CARLOS,FUFSCAR,2415338000130,CAMPUS SOROCABA,FUF-SAO CARLOS-SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,000082024,26417,ANTARES ENGENHARIA LTDA - 34.367.007/0001-68,MIN.EDUC,158471,INST. FED. DE EDUC. CIENC. E TEC. DA PARAIBA -...,IFPB/PRINCESA ISABE,34367007000168,CAMPUS-PI,IF DA PARAIBA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,000082024,41231,SEVEN CONSULTORIA E PROJETOS,MCOM,413012,AGENCIA NACIONAL DE TELECOMUNICACOES AM,ANATEL-AMAZONAS,18737991000155,ANATEL AM,ANATEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_contratos["nr_contrato"].isna().value_counts()

False    22055
Name: nr_contrato, dtype: int64

In [5]:
df_contratos["id_contrato"].isna().value_counts()

True     21451
False      604
Name: id_contrato, dtype: int64

In [6]:
df_contratos.columns

Index(['nr_contrato', 'cd_orgao_siafi', 'nm_razao_social',
       'sg_orgao_sup_tabela_ug', 'cd_ug_gestora', 'nm_ug_tabela_ug',
       'sg_ug_gestora', 'nr_cnpj', 'nm_unidade_prestacao', 'sg_orgao',
       'nm_orgao', 'cd_orgao_siape', 'cnpj_formatado', 'status_validacao',
       'id_contrato', 'objeto_contrato', 'objeto_compra', 'dataAssinatura',
       'dataPublicacaoDOU', 'dataInicioVigencia', 'dataFimVigencia',
       'tipo_fornecedor', 'valorInicialCompra', 'valorFinalCompra'],
      dtype='object')

## Agrupar os terceirizados por categoria profissional

In [7]:
df_total = pd.read_csv("../data/terceirizados/arquivos_tratados/df_total.csv", low_memory=False)

In [8]:
# Função para garantir que nr_contrato tenha exatamente 9 dígitos
def formatar_nr_contrato(nr_contrato):
    nr_contrato = str(nr_contrato).strip()  # Remover espaços extras

    # Se já tiver 9 dígitos, remover zeros à esquerda e preencher novamente para garantir 9 dígitos
    if len(nr_contrato) == 9:
        return nr_contrato.lstrip('0').zfill(9)
    
    # Se tiver menos de 9 dígitos, adicionar zeros à esquerda
    return nr_contrato.zfill(9)

# Aplicando a função à coluna nr_contrato
df_total['nr_contrato'] = df_total['nr_contrato'].apply(formatar_nr_contrato)

In [9]:
# Corrigir cd_orgao_siafi para MGI e ME dentro do filtro específico
df_total.loc[(df_total['nm_ug_tabela_ug'] == 'MINISTERIO DA ECONOMIA') & 
              (df_total['cd_orgao_siafi'] == -2) & 
              (df_total['sg_orgao_sup_tabela_ug'] == 'MGI'), 'cd_orgao_siafi'] = 46000

# Por uma escolha metodológica, também vamos atribuir o código SIAFI do MGI, uma vez que o COLABORAGOV é um guarda-chuva para mais de 12 ministérios.
# A chance de acerto aumenta, uma vez que é difícil identificar por nm_unidade_prestacao
df_total.loc[(df_total['nm_ug_tabela_ug'] == 'MINISTERIO DA ECONOMIA') & 
              (df_total['cd_orgao_siafi'] == -2) & 
              (df_total['sg_orgao_sup_tabela_ug'] == 'ME.'), 'cd_orgao_siafi'] = 46000

In [10]:
df_siafi_negativo = df_total[df_total['cd_orgao_siafi'] == -2].groupby('nm_ug_tabela_ug').size().reset_index(name='quantidade')

In [11]:
lista_siafi = [26405,
 26405,
 26432,
 26432,
 26432,
 26432,
 26405,
 26405,
 26432,
 26432,
 26432,
 26432,
 55000,
 26279,
 26280,
 26280,
 51000,
 26410,
 26410,
 26410,
 26417,
 26417,
 26409,
 26417,
 26417,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26421,
 26421,
 26439,
 26432,
 26432,
 26432,
 26432,
 26432,
 51000,
 51000,
 46000,
 46000,
 30000,
 30000,
 30000,
 26000,
 26000,
 30000,
 26455,
 26455]

In [12]:
df_siafi_negativo['cd_orgao_siafi_corrigido'] = lista_siafi

In [13]:
# Criar um dicionário de mapeamento
mapa_siafi = dict(zip(df_siafi_negativo['nm_ug_tabela_ug'], df_siafi_negativo['cd_orgao_siafi_corrigido']))

# Aplicar a correção no df_total
df_total.loc[df_total['cd_orgao_siafi'] == -2, 'cd_orgao_siafi'] = df_total['nm_ug_tabela_ug'].map(mapa_siafi)

In [14]:
def formatar_cnpj(cnpj):
    """Formata um CNPJ no padrão 'XX.XXX.XXX/XXXX-XX'"""
    cnpj = re.sub(r'\D', '', str(cnpj))  # Remove tudo que não é número
    cnpj = cnpj.zfill(14)  # Completa com zeros à esquerda se tiver menos de 14 dígitos
    if len(cnpj) == 14:
        return f"{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}"
    return ""  # Retorna vazio se não for um CNPJ válido

In [15]:
# Criar a nova coluna 'cnpj_formatado'
df_total['cnpj_formatado'] = df_total['nr_cnpj'].apply(formatar_cnpj)

In [16]:
df_total.columns

Index(['id_terc', 'sg_orgao_sup_tabela_ug', 'cd_ug_gestora', 'nm_ug_tabela_ug',
       'sg_ug_gestora', 'nr_contrato', 'nr_cnpj', 'nm_razao_social', 'nr_cpf',
       'nm_terceirizado', 'nm_categoria_profissional', 'nm_escolaridade',
       'nr_jornada', 'nm_unidade_prestacao', 'vl_mensal_salario',
       'vl_mensal_custo', 'Num_Mes_Carga', 'Mes_Carga', 'Ano_Carga',
       'sg_orgao', 'nm_orgao', 'cd_orgao_siafi', 'cd_orgao_siape',
       'cnpj_formatado'],
      dtype='object')

In [18]:
# Converter diretamente para float e limitar casas decimais
df_total['vl_mensal_salario'] = pd.to_numeric(df_total['vl_mensal_salario'], errors='coerce').round(2)
df_total['vl_mensal_custo'] = pd.to_numeric(df_total['vl_mensal_custo'], errors='coerce').round(2)

# Verificar se restaram nulos após a conversão
print("\n🔎 Valores ainda nulos após tentativa de correção:")
print(df_total[['vl_mensal_salario', 'vl_mensal_custo']].isnull().sum())

# Opcional: Preencher nulos com a mediana (mais seguro para não distorcer os valores)
df_total['vl_mensal_salario'].fillna(df_total['vl_mensal_salario'].median(), inplace=True)
df_total['vl_mensal_custo'].fillna(df_total['vl_mensal_custo'].median(), inplace=True)


🔎 Valores ainda nulos após tentativa de correção:
vl_mensal_salario    0
vl_mensal_custo      0
dtype: int64


In [19]:
# Identificar valores problemáticos na coluna 'nr_jornada'
valores_problematicos = df_total[~df_total['nr_jornada'].str.strip().str.isnumeric()]
print("Amostra de valores problemáticos:")
print(valores_problematicos[['nr_jornada']].head(10))

# Verificar se há espaços ou textos estranhos
print("\nValores únicos na coluna 'nr_jornada':")
print(df_total['nr_jornada'].unique())

🔎 Amostra de valores problemáticos:
      nr_jornada
6130        NI  
9867        NI  
10822       NI  
48101       NI  
60812       NI  
61289       NI  
64615       NI  
65003       NI  
69061       NI  
80888       NI  

🔎 Valores únicos na coluna 'nr_jornada':
['44  ' '40  ' '30  ' '20  ' '35  ' '48  ' '36  ' '41  ' '25  ' 'NI  '
 '45  ' '10  ' '11  ' '47  ' '16  ' '22  ' '42  ' '15  ' '46  ' '48' '44'
 '40' '41' '30' '36' '20' '25' '22' '42' '15' '35' '45' '10' '16' '46'
 '11' '49  ' '54  ' '53  ' '52  ' '51  ' '61  ' '59  ' '57  ' '58  '
 '56  ' '37  ' '39  ' '33  ' '27  ' '13  ' '32  ' '28  ' '18  ' '23  '
 '84  ' '31  ' '32' '37' '84' '26' '28' '23' '27']


In [20]:
# Limpeza e conversão
df_total['nr_jornada'] = (
    df_total['nr_jornada']
    .astype(str)         # Garante que tudo é string
    .str.strip()         # Remove espaços em branco
    .replace('NI', np.nan)  # Substitui "NI" por NaN para facilitar média e mediana
    .astype(float)       # Converte para float
)

In [21]:
# Agrupamento com agregações numéricas e colunas adicionais
df_agrupado_categoria = df_total.groupby(['nr_contrato', 'nm_categoria_profissional']).agg({
    'vl_mensal_salario': ['mean', 'median'],
    'vl_mensal_custo': ['mean', 'median'],
    'nr_jornada': ['mean', 'median'],  
    'id_terc': 'count',
    'sg_orgao_sup_tabela_ug': 'first',
    'cd_ug_gestora': 'first',
    'nm_ug_tabela_ug': 'first',
    'sg_ug_gestora': 'first',
    'nm_razao_social': 'first',
    'nm_unidade_prestacao': 'first',
    'sg_orgao': 'first',
    'nm_orgao': 'first',
    'cd_orgao_siafi': 'first',
    'cd_orgao_siape': 'first',
    'cnpj_formatado': 'first'
}).reset_index()

In [22]:
# Renomear colunas para remover multi-index gerado pelas agregações
df_agrupado_categoria.columns = ['_'.join(col).rstrip('_') for col in df_agrupado_categoria.columns]

In [23]:
df_agrupado_categoria.head()

,nr_contrato,nm_categoria_profissional,vl_mensal_salario_mean,vl_mensal_salario_median,vl_mensal_custo_mean,vl_mensal_custo_median,nr_jornada_mean,nr_jornada_median,id_terc_count,sg_orgao_sup_tabela_ug_first,cd_ug_gestora_first,nm_ug_tabela_ug_first,sg_ug_gestora_first,nm_razao_social_first,nm_unidade_prestacao_first,sg_orgao_first,nm_orgao_first,cd_orgao_siafi_first,cd_orgao_siape_first,cnpj_formatado_first
0,000002019,517330 - VIGILANTE,1390.400,1390.40,3688.5600,3688.56,44.0,44.0,1,MIN.EDUC,154042,UNIVERIDADE FEDERAL DO RIO GRANDE - FURG,FURG,SULCLEAN SERVICOS LTDA.,CIDEC / REITORIA,FURG,UNIVERSIDADE FEDERAL DO RIO GRANDE - FURG,26273,26273,06.205.427/0001-02
1,000002023,411010 - ASSISTENTE ADMINISTRATIVO,1000.000,1000.00,2000.0000,2000.00,40.0,40.0,1,MINC,343029,SUPERINTENDENCIA DO IPHAN NO DISTRITO FEDERAL,IPHAN-DF,XYZ LTDA,IPHAN - DF,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,20411,24204,00.000.000/0000-00
2,000012011,517330 - VIGILANTE,1482.845,1308.80,4773.4875,4320.12,41.0,41.0,16,MINIST.,343002,SUPERINTENDENCIA DO IPHAN NO PARA,IPHAN-PA,POLO SEGURANCA ESPECIALIZADA LTDA.,PREDIO SEDE/PA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,20411,24204,02.650.833/0001-23
3,000012012,"313115 - ELETROTECNICO NA FABRICACAO, MONTAGEM...",1549.110,1549.11,4281.8700,4281.87,44.0,44.0,1,MDS,512087,GERENCIA EXECUTIVA DUQUE DE CAXIAS,GEX/DUQUE DE CAXIAS,MITRA ENG E MONT.INDUST LTDA,APS DUQUE DE CAXIAS,INSS,INSTITUTO NACIONAL DO SEGURO SOCIAL,37202,57202,00.000.007/3678-13
4,000012012,514320 - FAXINEIRO,998.000,998.00,2692.4900,2692.49,40.0,40.0,8,MINIST.,343003,IPHAN - SUPERINTENDENCIA DO MARANHAO,IPHAN-MA,ARE SERVICOS LTDA,SUPERINTENDENCIA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,20411,24204,11.873.594/0001-61


In [24]:
df_agrupado_categoria.shape

(24281, 20)

## Juntar os dataframes para que as categorias tenham os dados do respectivo contrato

In [25]:
# Renomear colunas para corresponder ao df_contratos
df_agrupado_categoria.rename(columns={
    'cd_orgao_siafi_first': 'cd_orgao_siafi',
    'nm_razao_social_first': 'nm_razao_social',
    'sg_orgao_sup_tabela_ug_first': 'sg_orgao_sup_tabela_ug',
    'cd_ug_gestora_first': 'cd_ug_gestora',
    'nm_ug_tabela_ug_first': 'nm_ug_tabela_ug',
    'sg_ug_gestora_first': 'sg_ug_gestora',
    'nm_unidade_prestacao_first': 'nm_unidade_prestacao',
    'sg_orgao_first': 'sg_orgao',
    'nm_orgao_first': 'nm_orgao',
    'cd_orgao_siape_first': 'cd_orgao_siape',
    'cnpj_formatado_first': 'cnpj_formatado'
}, inplace=True)

In [26]:
# Padronizar colunas para string nos dois dataframes
colunas_chave = ['nr_contrato', 'cd_orgao_siafi', 'nm_razao_social']

for col in colunas_chave:
    df_agrupado_categoria[col] = df_agrupado_categoria[col].astype(str).str.strip()
    df_contratos[col] = df_contratos[col].astype(str).str.strip()

In [27]:
# Preencher os valores do df_contratos no df_agrupado_categoria
df_agrupado_categoria = df_agrupado_categoria.merge(
    df_contratos[['nr_contrato', 'cd_orgao_siafi', 'nm_razao_social',
                   'sg_orgao_sup_tabela_ug', 'cd_ug_gestora', 'nm_ug_tabela_ug',
                   'sg_ug_gestora', 'nr_cnpj', 'nm_unidade_prestacao', 'sg_orgao',
                   'nm_orgao', 'cd_orgao_siape', 'cnpj_formatado', 'status_validacao',
                   'id_contrato', 'objeto_contrato', 'objeto_compra', 'dataAssinatura',
                   'dataPublicacaoDOU', 'dataInicioVigencia', 'dataFimVigencia',
                   'tipo_fornecedor', 'valorInicialCompra', 'valorFinalCompra']],
    on=['nr_contrato', 'cd_orgao_siafi', 'nm_razao_social'],
    how='left'
)

In [28]:
# Filtrar apenas as linhas que possuem id_contrato preenchido
df_agrupado_filtrado = df_agrupado_categoria.dropna(subset=['id_contrato'])

In [29]:
df_agrupado_filtrado.head()

,nr_contrato,nm_categoria_profissional,vl_mensal_salario_mean,vl_mensal_salario_median,vl_mensal_custo_mean,vl_mensal_custo_median,nr_jornada_mean,nr_jornada_median,id_terc_count,sg_orgao_sup_tabela_ug_x,...,id_contrato,objeto_contrato,objeto_compra,dataAssinatura,dataPublicacaoDOU,dataInicioVigencia,dataFimVigencia,tipo_fornecedor,valorInicialCompra,valorFinalCompra
5,000012012,517330 - VIGILANTE,1775.28,1775.28,3923.92,3923.92,40.0,40.0,46,MINIST.,...,668336396.0,Objeto: Prestação de serviço de vigilancia d...,Objeto: Pregão Eletrônico - Contratação de em...,2012-06-18,2012-07-05,2012-06-18,2013-06-18,Entidades Empresariais Privadas,687542.0,2660693.36
36,000012014,513315 - CAMAREIRO DE HOTEL,1549.11,1549.11,4281.87,4281.87,44.0,44.0,4,MP.,...,668318167.0,Objeto: Contratação de empresa especializada n...,Objeto: Pregão Eletrônico - Contratação de em...,2009-01-12,2009-01-13,2009-01-12,2010-01-11,Entidades Empresariais Privadas,913810.56,5687893.2
43,000012014,514315 - LIMPADOR DE FACHADAS,1549.11,1549.11,4281.87,4281.87,44.0,44.0,1,MP.,...,668318167.0,Objeto: Contratação de empresa especializada n...,Objeto: Pregão Eletrônico - Contratação de em...,2009-01-12,2009-01-13,2009-01-12,2010-01-11,Entidades Empresariais Privadas,913810.56,5687893.2
51,000012014,622015 - TRABALHADOR NA PRODUCAO DE MUDAS E SE...,1549.11,1549.11,4281.87,4281.87,44.0,44.0,1,MP.,...,668318167.0,Objeto: Contratação de empresa especializada n...,Objeto: Pregão Eletrônico - Contratação de em...,2009-01-12,2009-01-13,2009-01-12,2010-01-11,Entidades Empresariais Privadas,913810.56,5687893.2
123,000012016,313120 - TECNICO DE MANUTENCAO ELETRICA,4131.04,4131.04,9030.21,9030.21,30.0,30.0,5,MCTI,...,668296909.0,Objeto: Prestação de serviços de Limpeza e Con...,"Objeto: Pregão Eletrônico - Contratação,em re...",2011-01-03,2011-02-11,2011-01-03,2012-01-03,Entidades Empresariais Privadas,764634.12,4775950.54


In [30]:
df_agrupado_filtrado.shape

(918, 41)

In [31]:
#df_agrupado_filtrado.to_csv('../data/contratos/categoria_contrato_v2.csv', index=False)